Le but de ce notebook est de déterminer s'il existe une question q1, extraite d'un database de questions sur le forum Quora, ressemblant à une autre question q2 présente dans le database.

# Importation de la base de données

In [1]:
!pip install -U sentence-transformers
!pip install numpy
!pip install pandas
!pip install python-magic-bin
!pip install ipywidgets

ERROR: Could not find a version that satisfies the requirement python-magic-bin (from versions: none)
ERROR: No matching distribution found for python-magic-bin


In [2]:
import numpy as np
import pandas as pd
import os

from google.colab import drive
drive.mount('/content/drive')

# unzip les données
!unzip -o '/content/drive/MyDrive/Colab Notebooks/dataset/quora_questions/quora-question-pairs.zip'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Archive:  /content/drive/MyDrive/Colab Notebooks/dataset/quora_questions/quora-question-pairs.zip
  inflating: sample_submission.csv.zip  
  inflating: test.csv                
  inflating: test.csv.zip            
  inflating: train.csv.zip           


In [3]:
df = pd.read_csv('train.csv.zip')
df.head(20)

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0
5,5,11,12,Astrology: I am a Capricorn Sun Cap moon and c...,"I'm a triple Capricorn (Sun, Moon and ascendan...",1
6,6,13,14,Should I buy tiago?,What keeps childern active and far from phone ...,0
7,7,15,16,How can I be a good geologist?,What should I do to be a great geologist?,1
8,8,17,18,When do you use シ instead of し?,"When do you use ""&"" instead of ""and""?",0
9,9,19,20,Motorola (company): Can I hack my Charter Moto...,How do I hack Motorola DCX3400 for free internet?,0


In [4]:
df_duplicates = df[df['is_duplicate'] == 1]
df_duplicates.head()

,id,qid1,qid2,question1,question2,is_duplicate
5,5,11,12,Astrology: I am a Capricorn Sun Cap moon and c...,"I'm a triple Capricorn (Sun, Moon and ascendan...",1
7,7,15,16,How can I be a good geologist?,What should I do to be a great geologist?,1
11,11,23,24,How do I read and find my YouTube comments?,How can I see all my Youtube comments?,1
12,12,25,26,What can make Physics easy to learn?,How can you make physics easy to learn?,1
13,13,27,28,What was your first sexual experience like?,What was your first sexual experience?,1


In [5]:
# Regrouper toutes les questions dans une seule liste
questions = list(set(  df['question1'] .str.lower() .unique() .tolist() + \
                       df['question2'] .str.lower() .unique() .tolist()  ))
print(len(questions))

537191


# Présentation du modèle utilisé

'all-distilroberta-v1'  donne de meilleurs résultats que 'all-mpnet-base-v2'

In [6]:
from sentence_transformers import SentenceTransformer
import torch

# Load https://huggingface.co/sentence-transformers/all-mpnet-base-v2
# essai 2 avec 'all-distilroberta-v1'

model_nom = 'all-distilroberta-v1'
model = SentenceTransformer(model_nom,
                            device = "cuda" if torch.cuda.is_available() else "cpu" )



/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavio

In [7]:
# # un vecteur représenatnt une question a une dimension de 768
# embeddings = model.encode(sub_questions,convert_to_numpy=True)
# print(embeddings.shape)

# # Matrice de similarité des questions => CUDA out of memory
# similarities = model.similarity(embeddings, embeddings)
# similarities

La matrice similarities nous donnes les ressemblances entre 2 questions. Cependant, comparer un à un les 537191 questions seraient trop chronophage  
On utilise alors la méthode FAISS (Facebook AI Similarity Search) pour ne déterminer que les k questions les plus ressemblantes, permettant d'économiser le temps de calcul pour les questions les plus éloignées

# Recherche de phrases similaires

Vectorisation de toutes les questions par le modèle


In [9]:
# Encodage des embeddings
embeddings_path = f'/content/drive/MyDrive/Colab Notebooks/embeddings_{model_nom}.npy'
forced = True

if not os.path.isfile(embeddings_path) or forced == False:
  # encodage en gpu
  embeddings = model.encode(questions,
                          batch_size=64,
                          convert_to_numpy=True,
                          show_progress_bar=True)

  # Sauvegarder les embeddings après le long calcul pour éviter l'encodage
  np.save(embeddings_path, embeddings) # prend 1.23GB

else:
  # chargement des embeddings depuis un fichier
  embeddings = np.load(embeddings_path)
  assert len(questions) == embeddings.shape[1], \
         "La taille des embeddings ne correspond pas à la longueur des questions."

<>:18: SyntaxWarning: assertion is always true, perhaps remove parentheses?
<>:18: SyntaxWarning: assertion is always true, perhaps remove parentheses?
<ipython-input-9-8f387fa6f252>:18: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert(len(questions) == embeddings.shape[1], "La taille des embeddings ne correspond pas à la longueur des questions.")


Batches:   0%|          | 0/8394 [00:00<?, ?it/s]

Calcul de similarité avec la comparaison du modèle

Cas 1 : 10 phrases prises au hasard dans le dataset de train

In [11]:
def find_similar_questions(question_cible, embeddings, nb_neighboors: int = 10):
  question_cible = question.lower()
  query_embedding = model.encode(question_cible,
                                  show_progress_bar=False,
                                  convert_to_tensor=True)

  device = 'cuda' if torch.cuda.is_available() else 'cpu'
  query_embedding = query_embedding.to(device)
  embeddings = torch.tensor(embeddings).to(device)


  # compare la question avec toues les autres
  similarity_scores = model.similarity(query_embedding, embeddings)[0]

  # renvoie un tensor contenant les scores et les indices des embeddings les plus proches
  scores, indices = torch.topk(similarity_scores, k=nb_neighboors)

  return scores, indices

In [13]:
nb_question_cibles = 10
row_random = df.sample(nb_question_cibles)

for question in row_random['question1']:
  # Trouver les questions similaires
  scores, indices = find_similar_questions(question, embeddings)

  # Boucle pour collecter les questions et scores
  l_results = []
  for  score,idx  in zip(scores[1:], indices[1:]):
    l_results.append([questions[idx], f"{score:.2f}"])

  # Afficher les résultats sous forme de DataFrame
  print(f'\n\n\n\n\n\n Query: {question} \
        \n\n\n MOST SIMILAR QUESTIONS:')
  display(pd.DataFrame(l_results, columns=['Questions', 'Similarity Score']))
  pd.set_option('display.max_colwidth', None)







 Query: What does それでもまだ mean in English?         


 MOST SIMILAR QUESTIONS:


,Questions,Similarity Score
0,what is 睨んでるin english?,0.61
1,"what is the meaning of the spanish phrase ""que es eso""?",0.51
2,what does 'desu wa des' mean?,0.51
3,"what does the japanese phrase ""nande desu ka"" mean?",0.51
4,"""your best teacher is your last mistake."" what does it mean?",0.50
5,"what does ""donde esta"" mean?",0.50
6,"what is the meaning of the spanish phrase ""que mes es""?",0.50
7,"what does ""まな"" mean in japanese?",0.50
8,what does this phrase in english means?,0.49








 Query: What are the top 5 books you've read?         


 MOST SIMILAR QUESTIONS:


,Questions,Similarity Score
0,what are the 5 best books you've read?,0.97
1,what are the top 3 books you have read?,0.95
2,what are the top 5/ 10 books you've ever read?,0.92
3,what are your top 5 favorite books?,0.92
4,which is the top ten must read books you recommend?,0.91
5,what is the top ten must read books that you recommend?,0.91
6,what are your top ten favorite books?,0.91
7,what are your top five favorite books and why?,0.90
8,what are the top 10 books l must read?,0.89








 Query: Which is the most profitable product to import from China and sell in India?         


 MOST SIMILAR QUESTIONS:


...

,Questions,Similarity Score
0,what are the top 30 pearls of wisdom from sarvepalli radhakrishnan?,0.50
1,what is the most highly prized sapphire?,0.48
2,what is phi sigma pi?,0.46
3,what is the target market for pearl jewelry?,0.46
4,what are the unlimited rare candies in pokemon emerald?,0.45
5,how can i do pearl farming?,0.45
6,what are the 4 pillars of oops?,0.45
7,truth table of pipo?,0.45
8,what are the top 30 pearls of wisdom from sarvapally radhakrishnan?,0.45








 Query: Why don't babies cry inside the womb?         


 MOST SIMILAR QUESTIONS:


,Questions,Similarity Score
0,why don't babies cry in a mothers womb?,0.96
1,why do babies cry when they're born?,0.84
2,is it possible for babies to cry in the womb?,0.83
3,why do babies cry?,0.82
4,why do babies cry when they are born? do all mammals cry when they are born?,0.82
5,why do some babies not cry just after birth and what is the problem behind it?,0.81
6,why do babies cry soon after they are delivered? what is the scientific reason behind it?,0.76
7,why does my baby never cry?,0.74
8,why do babies cry at night?,0.69








 Query: Will time travel ever become possable?         


 MOST SIMILAR QUESTIONS:


,Questions,Similarity Score
0,can time travel ever be possible?,0.90
1,will time travel be possible in future?,0.90
2,will we ever have the technology to time travel?,0.89
3,can we expect time travel to become a reality?,0.89
4,is time traveling possible in future?,0.88
5,is time travel possible?,0.88
6,will time travel be possible in the next 10 years?,0.88
7,do you think time travel is possible?,0.87
8,is time travel possible? will we ever be able to build a time machine?,0.87


Cas 2 : On regarde si dans les phrases dont on est surs qu'elles sont dupliqués, le modèle arrive à la même conclusion

In [14]:
nb_question_cibles = 10
row_random = df_duplicates.sample(nb_question_cibles)

for question in row_random['question1']:
  # Trouver les questions similaires
  scores, indices = find_similar_questions(question, embeddings)

  # Boucle pour collecter les questions et scores
  l_results = []
  for  score,idx  in zip(scores[1:], indices[1:]):
    l_results.append([questions[idx], f"{score:.2f}"])

  # Afficher les résultats sous forme de DataFrame
  print(f'\n\n\n\n\n\n Query: {question} \
        \n\n\n MOST SIMILAR QUESTIONS:')
  display(pd.DataFrame(l_results, columns=['Questions', 'Similarity Score']))
  pd.set_option('display.max_colwidth', None)







 Query: What makes The Hindu so special that it is the only newspaper which is recommended for UPSC aspirants?         


 MOST SIMILAR QUESTIONS:


,Questions,Similarity Score
0,what does hindu newspaper editorial gives a reader especially for upsc aspirants?,0.84
1,why is editorial page of the hindu so important for upsc aspirants?,0.84
2,which newspaper should one read for upsc - indian express or the hindu?,0.76
3,"which newspaper to be read, the hindu or the indian express for upsc?",0.75
4,how does the hindu newspaper collect news?,0.74
5,"between the indian express and the hindu, which one is a better newspaper? please support your answer with arguments",0.72
6,what are some of india's best newspapers and why?,0.71
7,what sections of newspaper (the hindu) are must to read daily from comptetion point of view?,0.68
8,how should i read the hindu newspaper?,0.68








 Query: Why do bad things seem to happen to good people?         


 MOST SIMILAR QUESTIONS:


,Questions,Similarity Score
0,why do bad things happen to good people?,0.95
1,why do bad things happen with good people?,0.94
2,why do bad things have to happen to good people?,0.93
3,why would bad things ever happen to good people?,0.92
4,"why do good things happen to bad people, and vice-versa?",0.92
5,why do evil things happen to good people?,0.91
6,why does good things happen to bad people and why bad things happen to good people?,0.86
7,why do bad things happen?,0.86
8,why do bad things happen to amazing people?,0.85








 Query: What should I read to start learning about quantum mechanics?         


 MOST SIMILAR QUESTIONS:


...

,Questions,Similarity Score
0,is pakistan really preparing for war with neighbor country india?,0.95
1,does india will start war against pakistan?,0.92
2,will india fight a war with pakistan?,0.91
3,is there going to be a war between india and pakistan?,0.91
4,will pakistan have a war with india?,0.91
5,is there going to be a war between india and pakistan in the near future?,0.90
6,"is war between india and pakistan,imminent?",0.90
7,will india fight a war against pakistan?,0.90
8,will india declare war against pakistan?,0.90








 Query: Does quantum mechanics exclude a deterministic world?         


 MOST SIMILAR QUESTIONS:


,Questions,Similarity Score
0,does quantum mechanics refute determinism?,0.80
1,is the universe deterministic?,0.73
2,does heisenberg's uncertainty principle actually defeat the concept of a deterministic future?,0.71
3,why is the many world theory deterministic?,0.68
4,why is the many worlds theory deterministic?,0.67
5,"why is quantum physics invoked to disprove determinism, when we have no control over quantum laws?",0.65
6,"is quantum physics truly unpredictable in behaviour, or is it that humans simply don't posses the devices to predict it accurately?",0.62
7,in quantum mechanics are nonlocality and probability directly related in the wavefunction?,0.59
8,is everything predetermined?,0.59








 Query: How can meritocracy breed intellectual elitism?         


 MOST SIMILAR QUESTIONS:


,Questions,Similarity Score
0,"does meritocracy breed intellectual elitism? if so, how?",0.95
1,why is intellectualism seen as inherently elitist?,0.71
2,what is the future of meritocracy?,0.67
3,is elitism a bad thing?,0.66
4,why is intelligence underestimated as a cause of success?,0.64
5,is meritocracy an essential part of capitalism?,0.64
6,is meritocracy better?,0.64
7,is meritocracy in silicon valley a myth?,0.61
8,elitism: is being poor and elitist mutually exclusive?,0.58


Conclusion :   
 - on observe que le modèle "comprend" relativement bien les phrases (càd sa vectorisation reflète bien le sens de la phrase) et que l'on peut se fier aux scores de similarité (on peut juger que les phrases avec un haut score se ressemblent)

 - on voit que les questions similaires ont un score au dessus de 0,9 en général. Il faudrait déterminer précisement ce seuil.

 - un fine-tuning du modèle propre à cette problèmatique est surement necéssaire pour obtenir une claire discrimination des questions dupliquées de celles qui sont uniques.